# **Relevancy score**

Authors : F.C, S.B

Date : January 13, 2021

**POC : depuis un data frame, si il y a des mots en commun entre les requêtes et le titre ou entre les requêtes et le résumé, les ajouter dans une nouvelle colonne de DataFrame**

In [55]:
!pip install deplacy
!python -m spacy download fr_core_news_sm

✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')


In [56]:
import pandas as pd
import pkg_resources,imp
imp.reload(pkg_resources)
import spacy
import re
import numpy as np
nlp=spacy.load("fr_core_news_sm")
import nltk
nltk.download('stopwords')
stopword=nltk.corpus.stopwords.words('french')
nltk.download('wordnet')
from nltk.stem.snowball import FrenchStemmer
stemmer = FrenchStemmer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [57]:
wn=nltk.WordNetLemmatizer()
def tokenize(text):
    tokens = re.split('\W+', text) #W+ means a word character or - can go there
    return tokens
def remove_stopwords(tokenzed_list):
    text=[word for word in tokenzed_list if word not in stopword]
    return text
def lemmatizing(tokenized_text):
    texte=[]
    for elem in tokenized_text:
      texte.append(stemmer.stem(elem))
    return texte
def return_mean_embedding(sentence):
    # Tokeniser la phrase
    doc = nlp(sentence)
    # Retourner la moyenne des vecteurs pour chaque phrase
    return np.mean([(X.vector) for X in doc], axis=0)

In [58]:
def return_word_embedding(sentence):
    # Tokeniser la phrase
    doc = nlp(sentence)
    # Retourner le vecteur lié à chaque token
    return [(X.vector) for X in doc]
def return_token(sentence):
    # Tokeniser la phrase
    doc = nlp(sentence)
    # Retourner le texte de chaque token
    return [X.text for X in doc]

In [59]:
df_test = pd.read_json('./df_crawling_clean.json', orient='split')

In [60]:
df_new=pd.read_json('./df_crawling_clean.json', orient='split')

In [61]:
#on a deux data frame, un qui fera les comparaisons un qui gardera les mots d'origine

df_new

,URL,Query,Title,Snippet,Position,Rank
0,https://reality.fr/realite-augmentee/,(réalité AROUND(2) augmentée AND trajectoire A...,Agence de Réalité Augmentée - Création d'Appli...,"""",NaN,1
1,https://www.ptc.com/fr/technologies/augmented-...,(réalité AROUND(2) augmentée AND trajectoire A...,Qu'est-ce que la réalité augmentée ? | PTC,Conséquence directe : elle connecte les employ...,NaN,3
2,https://www.adobe.com/fr/insights/5-realworld-...,(réalité AROUND(2) augmentée AND trajectoire A...,Cinq exemples de réalité augmentée innovants. ...,Les consommateurs étant de plus en plus récept...,NaN,4
3,https://cache.media.eduscol.education.fr/file/...,(réalité AROUND(2) augmentée AND trajectoire A...,Découvrir la réalité augmentée - media.eduscol...,La réalité augmentée (RA) ne doit pas être con...,NaN,5
4,https://corporate.apec.fr/home/nos-etudes/tout...,(réalité AROUND(2) augmentée AND trajectoire A...,La réalité augmentée / la réalité virtuelle – ...,Ces travaux s'inscrivent dans le cadre du proj...,NaN,6
...,...,...,...,...,...,...
90168,http://duspviz.mit.edu/tutorials/python-scraping/,(web AROUND(2) scraping AND AP),Web Scraping with Python - DUSPviz - MIT,In this class,NaN,96
90169,https://www.mrexcel.com/board/threads/web-scra...,(web AROUND(2) scraping AND AP),Web Scraping using VBA | MrExcel Message Board,Feb 4,NaN,97
90170,http://scrapingauthority.com/2016/09/07/xpath-...,(web AROUND(2) scraping AND AP),How to Write the Best XPATH and CSS Selectors ...,Sep 7,NaN,98
90171,https://www.niemanlab.org/2017/05/the-ap-style...,(web AROUND(2) scraping AND AP),The AP Stylebook now includes new guidelines o...,May 31,NaN,99


In [62]:
df_test['Title'] = [x.lower() for x in df_test.Title]

In [63]:
for i in range (len(df_test)):
  df_test['Query'][i] = df_test['Query'][i].lower()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [64]:
import nltk
nltk.download('stopwords')
import pandas as pd
import pickle
import time
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('french'))
from nltk.stem import LancasterStemmer
import numpy as np
import time

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [65]:
from nltk.stem import SnowballStemmer
def cleandesc(desc):
    sent = desc
    sent = "".join([x.lower() if x.isalpha()  else " " for x in sent])
    Porter=SnowballStemmer('french')
    sent = " ".join([Porter.stem(x) if x.lower() not in stop_words  else "" for x in sent.split()])
    sent = " ".join(sent.split())
    return sent
start_time=time.time()

#On prends la racine des mots pour le data frame df_test pour faire nos tests de comparaison

df_test['Title']= [cleandesc(x.Title) for x in df_test.itertuples()]
df_test['Snippet']= [cleandesc(x.Snippet) for x in df_test.itertuples()]
df_test['Query']= [cleandesc(x.Query) for x in df_test.itertuples()]
end_time=time.time()
print("total time : {} mn".format((end_time-start_time)/60))

total time : 0.6333176374435425 mn


In [66]:
for i in range(len(df_test)):
  df_test['Query'][i]=df_test['Query'][i].replace(" around","")
df_test

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,URL,Query,Title,Snippet,Position,Rank
0,https://reality.fr/realite-augmentee/,réalit augment and trajectoir financi,agenc réalit augment création appliqu ar,,NaN,1
1,https://www.ptc.com/fr/technologies/augmented-...,réalit augment and trajectoir financi,réalit augment ptc,conséquent direct connect emploi amélior résul...,NaN,3
2,https://www.adobe.com/fr/insights/5-realworld-...,réalit augment and trajectoir financi,cinq exempl réalit augment innov adob,consomm plus plus récept expérient ra,NaN,4
3,https://cache.media.eduscol.education.fr/file/...,réalit augment and trajectoir financi,découvr réalit augment medi eduscol educ fr,réalit augment ra doit être confondu réalit vi...,NaN,5
4,https://corporate.apec.fr/home/nos-etudes/tout...,réalit augment and trajectoir financi,réalit augment réalit virtuel tendanc méti,traval inscrivent cadr projet def co développ ...,NaN,6
...,...,...,...,...,...,...
90168,http://duspviz.mit.edu/tutorials/python-scraping/,web scraping and ap,web scraping with python duspviz mit,in this class,NaN,96
90169,https://www.mrexcel.com/board/threads/web-scra...,web scraping and ap,web scraping using vba mrexcel messag board,feb,NaN,97
90170,http://scrapingauthority.com/2016/09/07/xpath-...,web scraping and ap,how to writ the best xpath and css selector fo...,sep,NaN,98
90171,https://www.niemanlab.org/2017/05/the-ap-style...,web scraping and ap,the ap stylebook now includ new guidelin dat,may,NaN,99


In [67]:
#On supprime les parenthèses de la requête et la met en minuscule pour le data frame qui servira à la sortie
for i in range(len(df_new)):
  df_new['Query'][i]=df_new['Query'][i].lower()
  df_new['Query'][i]=df_new['Query'][i].replace("(","")
  df_new['Query'][i]=df_new['Query'][i].replace(" around2","")
  df_new['Query'][i]=df_new['Query'][i].replace(")","")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [68]:
df_new

,URL,Query,Title,Snippet,Position,Rank
0,https://reality.fr/realite-augmentee/,réalité augmentée and trajectoire financière,Agence de Réalité Augmentée - Création d'Appli...,"""",NaN,1
1,https://www.ptc.com/fr/technologies/augmented-...,réalité augmentée and trajectoire financière,Qu'est-ce que la réalité augmentée ? | PTC,Conséquence directe : elle connecte les employ...,NaN,3
2,https://www.adobe.com/fr/insights/5-realworld-...,réalité augmentée and trajectoire financière,Cinq exemples de réalité augmentée innovants. ...,Les consommateurs étant de plus en plus récept...,NaN,4
3,https://cache.media.eduscol.education.fr/file/...,réalité augmentée and trajectoire financière,Découvrir la réalité augmentée - media.eduscol...,La réalité augmentée (RA) ne doit pas être con...,NaN,5
4,https://corporate.apec.fr/home/nos-etudes/tout...,réalité augmentée and trajectoire financière,La réalité augmentée / la réalité virtuelle – ...,Ces travaux s'inscrivent dans le cadre du proj...,NaN,6
...,...,...,...,...,...,...
90168,http://duspviz.mit.edu/tutorials/python-scraping/,web scraping and ap,Web Scraping with Python - DUSPviz - MIT,In this class,NaN,96
90169,https://www.mrexcel.com/board/threads/web-scra...,web scraping and ap,Web Scraping using VBA | MrExcel Message Board,Feb 4,NaN,97
90170,http://scrapingauthority.com/2016/09/07/xpath-...,web scraping and ap,How to Write the Best XPATH and CSS Selectors ...,Sep 7,NaN,98
90171,https://www.niemanlab.org/2017/05/the-ap-style...,web scraping and ap,The AP Stylebook now includes new guidelines o...,May 31,NaN,99


In [69]:
df_new['Words_find']=''

In [70]:
def clear_out(df,df_n) :
  """
  parameters : df (un data frame avec des mots tokenisé pour faire la comparaison)
              df_n (un data frame avec les mots d'origine pour les récupérer proprement)
  L'objectif est de transformer la colone words_find de df_n pour récuper les mots qui sont dans le titre et le resumé.
  Mais les comparaisons se feront sur les mots réduits de df
  """
  for j in range (len(df)): # pour chaque ligne du df,
    separator_or = list(df['Query'][j].split(' or ')) #on stocke tous les couples de la requete dans une liste separator_or
    sep_or= list(df_n['Query'][j].split(' or '))
    list_test=[] #liste pour stocker les mots en commun pour chaque ligne
    for k in range (len(separator_or)): #pour chacun de ces couples,s
      separator_and = list(separator_or[k].split(' and ')) #on stocke tous les mots du couple de la requete dans une liste separator_and
      sep_and = list(sep_or[k].split(' and '))
      for i in range (len(separator_and)): #pour tous les mots de la requete,
        if (df['Title'][j].find(separator_and[i]) != -1) : #on le cherche dans le titre, et si c'est présent...
          list_test.append(sep_and[i])
        if (df['Snippet'][j].find(separator_and[i]) != -1) : #ensuite, on le cherche dans le resumé, et si c'est présent...
          list_test.append(sep_and[i]) #... on le met dans une liste
          list_test = list(set(list_test))
      str = ' '.join(list_test) #...
    df_n['Words_find'][j]=list_test
  return(df_n)


In [71]:
dt = clear_out(df_test,df_new)
dt

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,URL,Query,Title,Snippet,Position,Rank,Words_find
0,https://reality.fr/realite-augmentee/,réalité augmentée and trajectoire financière,Agence de Réalité Augmentée - Création d'Appli...,"""",NaN,1,[réalité augmentée]
1,https://www.ptc.com/fr/technologies/augmented-...,réalité augmentée and trajectoire financière,Qu'est-ce que la réalité augmentée ? | PTC,Conséquence directe : elle connecte les employ...,NaN,3,[réalité augmentée]
2,https://www.adobe.com/fr/insights/5-realworld-...,réalité augmentée and trajectoire financière,Cinq exemples de réalité augmentée innovants. ...,Les consommateurs étant de plus en plus récept...,NaN,4,[réalité augmentée]
3,https://cache.media.eduscol.education.fr/file/...,réalité augmentée and trajectoire financière,Découvrir la réalité augmentée - media.eduscol...,La réalité augmentée (RA) ne doit pas être con...,NaN,5,[réalité augmentée]
4,https://corporate.apec.fr/home/nos-etudes/tout...,réalité augmentée and trajectoire financière,La réalité augmentée / la réalité virtuelle – ...,Ces travaux s'inscrivent dans le cadre du proj...,NaN,6,[réalité augmentée]
...,...,...,...,...,...,...,...
90168,http://duspviz.mit.edu/tutorials/python-scraping/,web scraping and ap,Web Scraping with Python - DUSPviz - MIT,In this class,NaN,96,"[web scraping, ap]"
90169,https://www.mrexcel.com/board/threads/web-scra...,web scraping and ap,Web Scraping using VBA | MrExcel Message Board,Feb 4,NaN,97,"[web scraping, ap]"
90170,http://scrapingauthority.com/2016/09/07/xpath-...,web scraping and ap,How to Write the Best XPATH and CSS Selectors ...,Sep 7,NaN,98,[]
90171,https://www.niemanlab.org/2017/05/the-ap-style...,web scraping and ap,The AP Stylebook now includes new guidelines o...,May 31,NaN,99,[ap]


In [73]:
dt.to_json('df_words_find.json')